## import packages

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
import keras
from keras.models import Sequential, Model
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
from keras.layers import Dense, Dropout, Reshape, Flatten, Activation, Add, Input
from keras.layers.normalization import BatchNormalization

import tensorflow as tf
from sklearn.metrics import roc_auc_score

c:\users\xinyi\anaconda3\envs\keras\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## prepare notebook

In [3]:
# config project folder
os.chdir('../')

In [4]:
# config notebook display
%matplotlib inline
pd.set_option("display.max_rows", None)
pd.set_option('display.max_columns', None)

## read pre-processed data

In [5]:
# load pre-processed data for features and labels
features = np.load('data/safety/features_arr.npy')
labels = np.load('data/safety/labels_arr.npy')

In [6]:
N = features.shape[0] # number of bookings (records)
T = features.shape[1] # number of time slices (each time slice is a second)
C = features.shape[2] # number of features (channels)

print(N, T, C)

19968 3600 9


## build resnet model

In [7]:
def res_block(X,f,k,n):
    X_shortcut = X
    
    X = Conv1D(filters=f,kernel_size=k,padding='same',name =n+'conv'+str(1))(X)
    X = BatchNormalization(name=n+'batch'+str(1))(X)
    X = Activation('relu')(X)
    
    X = Conv1D(filters=f,kernel_size=k,padding='same', name=n+'conv'+str(2))(X)
    X = BatchNormalization(name=n+'batch'+str(2))(X)
    
    X  = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [8]:
def ResNet(input_shape):
    X_input = Input(input_shape)
    X = Conv1D(20, 5, activation='relu',padding='same', name='conv1')(X_input)
    X = MaxPooling1D(3)(X)
    X = Conv1D(40, 5, activation='relu',padding='same',name='conv2')(X)
    X = res_block(X,40,5,'res11')
    X = res_block(X,40,5,'res12')
    X = MaxPooling1D(3)(X)
    X = Conv1D(80, 5, activation='relu',padding='same',name='conv3')(X)
    X = res_block(X,80,5,'res21')
    X = res_block(X,80,5,'res22')
    X = MaxPooling1D(4)(X)
    X = Conv1D(160, 5, activation='relu',padding='same',name='conv4')(X)
    X = res_block(X,160,5,'res31')
    X = res_block(X,160,5,'res32')
    X = MaxPooling1D(4)(X)
    X = Conv1D(200, 5, activation='relu',padding='same',name='conv5')(X)
    X = res_block(X,200,5,'res41')
    X = res_block(X,200,5,'res42')
    X = MaxPooling1D(2)(X)
    X = Flatten()(X)
    X = Dropout(0.5)(X)
    X = Dense(40,activation='relu')(X)
    X = Dense(1,activation='sigmoid')(X)
    model = Model(inputs = X_input, outputs = X)
    return model

In [9]:
model = ResNet((T,C))
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 3600, 9)      0                                            
__________________________________________________________________________________________________
conv1 (Conv1D)                  (None, 3600, 20)     920         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling1d_1 (MaxPooling1D)  (None, 1200, 20)     0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv1D)                  (None, 1200, 40)     4040        max_pooling1d_1[0][0]            
__________________________________________________________________________________________________
res11conv1

In [10]:
# define AUC ROC as a metrics
def auroc(y_true, y_pred):
    return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)

In [11]:
# compile restnet
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', auroc])

## model training

In [12]:
BATCH_SIZE = 100
EPOCHS = 100

In [13]:
x_train = features
y_train = labels

In [14]:
callbacks_list = [
    keras.callbacks.EarlyStopping(monitor='val_auroc', mode='max', verbose=1, patience=10),
    keras.callbacks.ModelCheckpoint(
        filepath='model/best_model.{epoch:02d}-{val_auroc:.2f}.h5',
        monitor='val_auroc',mode='max', save_best_only=True)
    ]

In [15]:
history = model.fit(x_train,
                    y_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    callbacks=callbacks_list,
                    validation_split=0.2,
                    verbose=1)

Train on 15974 samples, validate on 3994 samples
Epoch 1/100
15974/15974 [==============================] - 30s 2ms/step - loss: 0.5871 - acc: 0.7562 - auroc: 0.6514 - val_loss: 0.5437 - val_acc: 0.7516 - val_auroc: 0.6879
Epoch 2/100
15974/15974 [==============================] - 20s 1ms/step - loss: 0.5246 - acc: 0.7624 - auroc: 0.6812 - val_loss: 0.5313 - val_acc: 0.7692 - val_auroc: 0.6916
Epoch 3/100
15974/15974 [==============================] - 20s 1ms/step - loss: 0.5146 - acc: 0.7651 - auroc: 0.6895 - val_loss: 0.5154 - val_acc: 0.7619 - val_auroc: 0.7039
Epoch 4/100
15974/15974 [==============================] - 20s 1ms/step - loss: 0.5072 - acc: 0.7671 - auroc: 0.7064 - val_loss: 0.5211 - val_acc: 0.7641 - val_auroc: 0.6917
Epoch 5/100
15974/15974 [==============================] - 20s 1ms/step - loss: 0.4996 - acc: 0.7713 - auroc: 0.7147 - val_loss: 0.4880 - val_acc: 0.7767 - val_auroc: 0.7269
Epoch 6/100
15974/15974 [==============================] - 20s 1ms/step - loss: 0